# 1. user별 사진을 보관할 수있는 dB 만들기

## 1.1 models.py 사용하지 않고 table만들기


> 필요한 필드
- 고유번호
- 유저명
- 이미지 경로명:이미지만 들어가는것이 좋음

### 1.1.1sqlite browser에서 table 만들기

In [1]:
'''
CREATE TABLE myboard_image
(
id INTEGER PRIMARY KEY AUTOINCREMENT,
author_id INTEGER NOT NULL,
filename VARCHAR(500) NOT NULL,
FOREIGN KEY(author_id) REFERENCES auth_user(id)
)

'''

'\nCREATE TABLE myboard_image\n(\nid INTEGER PRIMARY KEY AUTOINCREMENT,\nauthor_id INTEGER NOT NULL,\nfilename VARCHAR(500) NOT NULL,\nFOREIGN KEY(author_id) REFERENCES auth_user(id)\n)\n\n'

#### 1.1.2 models.py에서는

In [2]:
class Image(models.Model):
    author = models.ForeignKey('auth.User', on_delete = models.CASCADE)
    filename = models.CharField(max_length = 50)

NameError: name 'models' is not defined

### rock이 걸린 경우
- sqlite: 서버 시스템이 아님.
- 하나의 파일개념
- 사용하면 다른데서는 실행이 안됨.
- cursor로 하며 됨

#### 1.1.2 connection이용하면

In [ ]:
sql = '''
CREATE TABLE myboard_image
(
id INTEGER PRIMARY KEY AUTOINCREMENT,
author_id INTEGER NOT NULL,
filename VARCHAR(500) NOT NULL,
FOREIGN KEY(author_id) REFERENCES auth_user(id)
)

'''

cursor = connection.cursor()
cursor.execute(sql)



## 1.2 data insert

> sqlite brower에서
데이터 보기->새 레코드 추가가능

In [3]:
from django.contrib.auth.models import User
from myboard import models
from django.db import connection  # db를 연계할 수 있음.
from django.shortcuts import render
from django.test import RequestFactory


#### 실제로 데이터를 넣을 때는 username으로 찾아서, 저장해야 함.


In [4]:
# user의 id를 찾아서 author id와 file name을 넣어야 함.
username = 'lee'
sql = f"select id from auth_user where username = '{username}'"

cursor = connection.cursor()
cursor.execute(sql)
author_id = cursor.fetchone()[0]
author_id

2

In [5]:
filename = 'face2.png'

sql = f"""
INSERT INTO myboard_image
(author_id, filename)
VALUES ({author_id}, '{filename}');
"""

print(sql)


INSERT INTO myboard_image
(author_id, filename)
VALUES (2, 'face2.png');



In [6]:
v = cursor.execute(sql)

# 2. html page 만들기

In [7]:
sql = f'''
select filename
from myboard_image
where author_id = (select id from auth_user where username = '{username}')

'''
print(sql)


select filename
from myboard_image
where author_id = (select id from auth_user where username = 'lee')




In [8]:
def dectfetchall(cursor):
    desc = cursor.description
    return[
        dict(zip([col[0] for col in desc], row))
        for row in cursor.fetchall()
    ]
    

In [9]:
cursor = connection.cursor()
cursor.execute(sql)

data = dectfetchall(cursor)
context = {'data': data, 'username' : username}

print(context)

{'data': [{'filename': 'face1.png'}, {'filename': 'face2.png'}, {'filename': 'face4.png'}, {'filename': 'face3.png'}, {'filename': 'face5.png'}, {'filename': 'wood.jpg'}, {'filename': 'face2.png'}], 'username': 'lee'}


In [10]:
rf= RequestFactory()
url = '/upload'
request = rf.get(url)

In [11]:
response = render(request, 'myboard/photolist.html', context)

In [12]:
response.content.decode()

'\n\nlee   <br>\n    \n<!--  name이 있는 것만 서버로 전송됨 -->\n<form action="/upload" enctype="multipart/form-data" method="POST">\n    <input type=\'hidden\' name=\'csrfmiddlewaretoken\' value=\'13BAmu3Wfi359Z8DM6EM4awxUdqlYobD8bknNTcq3Uw3XkRXI4ULD4qPFLmk1yWw\' />\n    <!--  바이너리 형태로 파일 전송 -->\n        <input type="file" name="filename">\n        <input type="submit" value="사진 전송">\n        <input type =\'hidden\' value = \'lee\' name=\'username\'>\n</form>\n\n\n\n\n\n    <img src="/static/face/lee/face1.png" width = \'100\'>\n    \n\n\n    <img src="/static/face/lee/face2.png" width = \'100\'>\n    \n\n\n    <img src="/static/face/lee/face4.png" width = \'100\'>\n    \n\n\n    <img src="/static/face/lee/face3.png" width = \'100\'>\n    \n\n\n    <img src="/static/face/lee/face5.png" width = \'100\'>\n    \n\n\n    <img src="/static/face/lee/wood.jpg" width = \'100\'>\n    \n\n\n    <img src="/static/face/lee/face2.png" width = \'100\'>\n    \n\n'

In [13]:



sql = f'''
select filename
from myboard_image
where author_id = (select id from auth_user where username = '{username}')
'''

cursor = connection.cursor()
cursor.execute(sql)

data = dectfetchall(cursor)
context = {'data': data, 'username' : username}

render(request, 'myboard/photolist.html', context)

<HttpResponse status_code=200, "text/html; charset=utf-8">

In [14]:
print(response.content.decode())



lee   <br>
    
<!--  name이 있는 것만 서버로 전송됨 -->
<form action="/upload" enctype="multipart/form-data" method="POST">
    <input type='hidden' name='csrfmiddlewaretoken' value='13BAmu3Wfi359Z8DM6EM4awxUdqlYobD8bknNTcq3Uw3XkRXI4ULD4qPFLmk1yWw' />
    <!--  바이너리 형태로 파일 전송 -->
        <input type="file" name="filename">
        <input type="submit" value="사진 전송">
        <input type ='hidden' value = 'lee' name='username'>
</form>





    <img src="/static/face/lee/face1.png" width = '100'>
    


    <img src="/static/face/lee/face2.png" width = '100'>
    


    <img src="/static/face/lee/face4.png" width = '100'>
    


    <img src="/static/face/lee/face3.png" width = '100'>
    


    <img src="/static/face/lee/face5.png" width = '100'>
    


    <img src="/static/face/lee/wood.jpg" width = '100'>
    


    <img src="/static/face/lee/face2.png" width = '100'>
    




In [15]:
# photolist.html
'''
{{username}}   <br>
    

<form action="/upload" enctype="multipart/form-data" method="POST">
    {% csrf_token %}
        <input type="file" name="filename">
        <input type="submit" value="사진 전송">
        <input type ='hidden' value = '{{username}}' name='username'>
</form>



{% for d in data %}

    <img src="/static/face/{{username}}/{{d.filename}}" width = '100'>
    
{% endfor %}


'''

'\n{{username}}   <br>\n    \n\n<form action="/upload" enctype="multipart/form-data" method="POST">\n    {% csrf_token %}\n        <input type="file" name="filename">\n        <input type="submit" value="사진 전송">\n        <input type =\'hidden\' value = \'{{username}}\' name=\'username\'>\n</form>\n\n\n\n{% for d in data %}\n\n    <img src="/static/face/{{username}}/{{d.filename}}" width = \'100\'>\n    \n{% endfor %}\n\n\n'

In [16]:
# views.py
'''

def photolist(request):
    username = 'lee'
    sql = f"select id from auth_user where username = '{username}'"

    cursor = connection.cursor()
    cursor.execute(sql)
    author_id = cursor.fetchone()[0] 

    sql = f'''
    select filename
    from myboard_image
    where author_id = (select id from auth_user where username = '{username}')
    '''

    cursor = connection.cursor()
    cursor.execute(sql)

    data = dictfetchall(cursor)
    context = {'data': data, 'username' : username}

    return render(request, 'myboard/photolist.html', context)


'''

IndentationError: unexpected indent (<ipython-input-16-149f880160e5>, line 13)

In [ ]:
# urls.py
'''
from django.urls import path
from . import views 
from django.shortcuts import redirect

urlpatterns = [
    # path('', views.index),
    path('', views.index),
    path('ajaxdel', views.ajaxdel),
    path('ajaxget', views.ajaxget),
    path('photolist', views.photolist),


    path('<category>/<int:pk>/<mode>', views.BoardView.as_view(), name = 'myboard'),
    # path('', lambda request: redirect('myboard', 'common', 0, 'list'),
    path('<category>/<int:page>', views.listsql),

    ]


'''



# 3. upload

1. db에서 insert해기
2. 실제 물리적인 폴더에도 넣기
3. 마지막에는 redirecr하기

## 3.1 upload
- pose 방식으로 변환해주기

#### shell-plus에서 post 방식을 쓰기 위해서 한 코드

In [ ]:
from django.core.files.uploadedfile import SimpleUploadedFile

In [ ]:
# 내 폴더 내 이미지 갖고오기
filename = 'wood.jpg'
# username = request.POST['username']


f = open(f'static/{filename}', 'rb')

file_upload= SimpleUploadedFile(filename,
                               f.read(),
                               content_type = 'image/jpeg')

In [ ]:
file_upload

In [ ]:
username = 'lee'
# post로 보내야 하는 namevalue
data = {'filename': file_upload,
       'username': username}

# 포스트 방식의 request 객체 만들기
request = rf.post('/myboard/upload', data = data, format = 'multipart')

In [ ]:
print(request)

In [ ]:
file = request.FILES['filename']
filename = file._name
print(filename)

In [ ]:
# 저장하기
fp = open(settings.BASE_DIR + '/static/face/' + username +'/'+ filename, 'wb')
for chunk in file.chunks():
    fp.write(chunk)
fp.close()

## 3.2. db insert

In [ ]:
# username을 이용해서 user id갖고오기
username = 'lee'
sql = f"select id from auth_user where username = '{username}'"

cursor = connection.cursor()
cursor.execute(sql)
author_id = cursor.fetchone()[0]
author_id

In [ ]:
sql = f"""
INSERT INTO myboard_image
(author_id, filename)
VALUES ({author_id}, '{filename}');
"""

print(sql)

In [ ]:
v = cursor.execute(sql)